In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
 shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [5]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [6]:
 max_id = len(tokenizer.word_index) # number of distinct characters
 dataset_size = encoded.shape[0] # total number of characters

In [7]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
n_steps = 100
window_length = n_steps + 1

In [9]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [10]:
mirrored_strategy=tf.distribute.MirroredStrategy()

In [11]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [12]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [13]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=100,
                    callbacks=[ResetStatesCallback()])

Epoch 1/100
313/313 [==============================] - 13s 20ms/step - loss: 2.6187
Epoch 2/100
313/313 [==============================] - 5s 17ms/step - loss: 2.2480
Epoch 3/100
313/313 [==============================] - 6s 18ms/step - loss: 2.1156
Epoch 4/100
313/313 [==============================] - 5s 17ms/step - loss: 2.0391
Epoch 5/100
313/313 [==============================] - 6s 18ms/step - loss: 1.9863
Epoch 6/100
313/313 [==============================] - 6s 18ms/step - loss: 1.9490
Epoch 7/100
313/313 [==============================] - 5s 17ms/step - loss: 1.9202
Epoch 8/100
313/313 [==============================] - 6s 18ms/step - loss: 1.8985
Epoch 9/100
313/313 [==============================] - 5s 17ms/step - loss: 1.8812
Epoch 10/100
313/313 [==============================] - 6s 18ms/step - loss: 1.8657
Epoch 11/100
313/313 [==============================] - 6s 20ms/step - loss: 1.8525
Epoch 12/100
313/313 [==============================] - 6s 18ms/step - loss: 1.8418


In [14]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [15]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [16]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [17]:
def preprocess(texts):
 X = np.array(tokenizer.texts_to_sequences(texts)) - 1
 return tf.one_hot(X, max_id)

In [18]:
def next_char(text, temperature=1):
 X_new = preprocess([text])
 y_proba = model.predict(X_new)[0, -1:, :]
 rescaled_logits = tf.math.log(y_proba) / temperature
 char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
 return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [19]:
def complete_text(text, n_chars=50, temperature=1):
 for _ in range(n_chars):
     text += next_char(text, temperature)
 return text

In [20]:
tf.random.set_seed(42)

print(complete_text("t"))

1/1 [==============================] - 0s 21ms/step
ther?

duke vincentio:
tut will he bear the oneton 


In [21]:
print(complete_text("Friends,Romans,Countrymen!",n_chars=500))

1/1 [==============================] - 0s 30ms/step
Friends,Romans,Countrymen! you broke tay endreace 
shopry to-old tritures alpent men law.
i patize you bear me, his rule-days his dangerer,
and he no sass thy carl to be mistoe as so old orared.

isabella:
thou inders the poor foots conment,
came with hit that i man: nevens with his
actions bin of chischard, at sweet sin!

slyowind, what but to? where i were come obe close.

leontes:
sirrah; let mome to thee, he rest and sweet,
and so tree! god's new: beseech it. i, and do the
besigft; all brun han well you. my lord, kno


In [22]:
#print(complete_text("A Scottish Lass!",n_chars=50000))

In [23]:
#print(complete_text("Morning on London Bridge,",n_chars=720))

In [24]:
#print(complete_text("Macbeth,",n_chars=720))

In [25]:
#print(complete_text("Ceasar:,",n_chars=5000))

In [26]:
#print(complete_text("2099,The World is,",n_chars=5000))